# Initial Insights
Notebook to derive some initial insights from the [@mediapermata](https://www.instagram.com/mediapermata/) Instagram page (Brunei's premier newspaper) with a particular focus on the comments.

In [1]:
# Load in necessary libraries
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
import malaya
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils

In [2]:
base_url = 'https://instagram.com'
user_name = utils.username
pass_word = utils.password
profile = 'mediapermata'

path = r'/Users/chong02/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get('https://www.instagram.com/')

# Logging in
time.sleep(10)
username=driver.find_element(By.NAME, 'username')
password=driver.find_element(By.NAME, 'password')
username.send_keys(user_name)
password.send_keys(pass_word)
password.send_keys(Keys.RETURN)

# Bypass save log in info
time.sleep(10)
not_now = driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()

# Bypass notifs
time.sleep(30)
not_now_again = driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()

# Search button
time.sleep(5)
open_search_tab = driver.find_element(By.XPATH, "//a[@href='#'][@role='link']").click()

# Search for profile
time.sleep(5)
search_bar = driver.find_element(By.XPATH, "//input[@placeholder='Search']")
search_bar.send_keys(profile)

# Click on profile
time.sleep(20)
profile_link = driver.find_element(By.XPATH, f"//a[@href='/{profile}/'][@role='link']").click()

# Infinite scrolling mechanism
time.sleep(10)
SCROLL_PAUSE_TIME = 1
counter = 0
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
while True and counter < 100:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    counter += 1
    
# Save page html
time.sleep(30)  # Wait for DOM to load
html = driver.page_source

In [3]:
# Create soup
soup = BeautifulSoup(html, 'html.parser')

In [21]:
def find_posts(href):
    return href and re.compile(r'\/p\/.+\/').search(href)

posts = soup.find_all(href=find_posts)
post_urls = [base_url + post.get('href') for post in posts]
print(f'{len(post_urls)} posts captured')

42 posts captured


In [18]:
def find_usernames(alt):
    return alt and re.compile(r'profile picture').search(alt)

post_comments = np.array([])
post_comments_users = np.array([])

for url in post_urls:
    driver.get(url)
    time.sleep(20)
    post_html = driver.page_source
    post_soup = BeautifulSoup(post_html, 'html.parser')
    comments = post_soup.find_all('span', class_='_aacl _aaco _aacu _aacx _aad7 _aade')
    comment_count = len(comments)
    users = post_soup.find_all(alt=find_usernames)[-comment_count:]
    comments_arr = np.array([])
    users_arr = np.array([])
    for comment in comments:
        text = comment.text
        comments_arr = np.append(comments_arr, text)
    for user in users:
        username = user.get('alt').split("'")[0]
        users_arr = np.append(users_arr, username)
    post_comments = np.append(post_comments, comments_arr)
    post_comments_users = np.append(post_comments_users, users_arr)

NameError: name 'posts_comments_users' is not defined

In [22]:
len(post_comments_users)

196

In [16]:
users[-3:][0].get('alt').split("'")[0]

'bibsterphs'

In [26]:
test1 = np.array(['hola', 'ni hao'])
test2 = np.array(['hello', 'helllo'])
combined = []
combined

TypeError: only integer scalar arrays can be converted to a scalar index